In [1]:
%matplotlib notebook
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
from scipy import signal
from ipyparallel import Client
from ipyparallel import depend, require, dependent
import imreg_dft as ird
#Shape of the data
height = 256
width = 256

hdd_directory = "/media/gu4n1n3/Data2/"

#28.815 and 86.5576
#90.14531424656546
TRUE_FRAMERATE = 30

In [20]:
# AUTOMATIC EXPERIMENT SELECTOR
directory = "/media/gu4n1n3/Data2/sfn2017/"
experiment = "20170913/2/Videos/"
#mouse = "M201609336"
mouse = "M201609136"
data_files = []

import os
from os.path import join, getsize
for root, dirs, files in os.walk(directory+experiment):
    for file in files:
        if mouse in file:
            data_files.append(directory+experiment+file)
            
            
mouse_text_file = None
for root, dirs, files in os.walk(directory):
    for file in files:
        if mouse in file and ".txt" in file:
            mouse_text_file = directory+file


In [3]:
class Trial:
    def __init__(self, filename):
        self.filename = filename
        self.correct_trials = []
        self.baselines = []
        
    def __str__(self, ):
        return(self.filename+str(self.correct_trials)+str(self.baselines))

In [21]:
current_trial = None
good_trials = []
with open(mouse_text_file, 'r') as file:
    data = file.readlines()
    for line in data:
        line = line.replace('\n', '')
        line = line.split('\t')
        if line[0] == 'Movie':
            line[2] = line[2].replace(':', '/')
            if current_trial != None:
                if len(current_trial.correct_trials) > 0:
                    good_trials.append(current_trial) 
            current_trial = Trial(hdd_directory+line[2])
        elif line[0] == 'stim':
            if "GO=2" in line[2]:
                current_trial.correct_trials.append(float(line[1]))
                current_trial.baselines.append(float(line[2][-4:]))
                
print(len(good_trials))            

87


In [25]:
trial_frames = []
bottom_end = 100
top_end = 60
bool_have_template = False
#Threshold for which to tell if the mean changed due to lighting issues
threshold = 30
for trial in good_trials:
    frames = get_raw_frames(trial.filename, 1)
    for i, stim_end in enumerate(trial.correct_trials):
        stim_end = int(stim_end*TRUE_FRAMERATE)
        frames_of_trial = frames[stim_end-bottom_end:stim_end+top_end]
        
        mean_trial = np.mean(np.mean(frames_of_trial, axis=1), axis=1)
        if np.max(mean_trial)-np.min(mean_trial) <= threshold:
            #do alignment of these images.
            if not bool_have_template:
                template_image = frames_of_trial[bottom_end]
                bool_have_template = True
            else:
                print("Translating frames...")
                result = ird.translation(template_image, frames_of_trial[bottom_end])
                tvec = result['tvec']
                for i in range(len(frames_of_trial)):
                    frames_of_trial[i] = ird.transform_img(frames_of_trial[i], tvec=tvec)
            # Calculate baseline the -6 is to remove the 200ms of the stimulus
            baseline = np.mean(frames_of_trial[:bottom_end-6], axis=0)
            frames_of_trial = np.divide(np.subtract(frames_of_trial, baseline), baseline)
            trial_frames.append(frames_of_trial)
            
trial_frames = np.asarray(trial_frames, dtype=np.float32) 

print(trial_frames.shape)

(1325, 256, 256, 3)
Translating frames...


/home/gu4n1n3/Applications/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: RuntimeWarning: divide by zero encountered in true_divide
/home/gu4n1n3/Applications/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: RuntimeWarning: invalid value encountered in true_divide


(1318, 256, 256, 3)
Translating frames...
(1372, 256, 256, 3)
Translating frames...
(1341, 256, 256, 3)
Translating frames...
(1376, 256, 256, 3)
Translating frames...
Translating frames...
(1321, 256, 256, 3)
Translating frames...
Translating frames...
(1375, 256, 256, 3)
Translating frames...
Translating frames...
(1379, 256, 256, 3)
Translating frames...
Translating frames...
(1379, 256, 256, 3)
Translating frames...
Translating frames...
(1324, 256, 256, 3)
Translating frames...
(1339, 256, 256, 3)
Translating frames...
(1377, 256, 256, 3)
Translating frames...
(1374, 256, 256, 3)
Translating frames...
(1332, 256, 256, 3)
Translating frames...
Translating frames...
Translating frames...
(1379, 256, 256, 3)
Translating frames...
Translating frames...
(1380, 256, 256, 3)
Translating frames...
(1332, 256, 256, 3)
Translating frames...
Translating frames...
(1315, 256, 256, 3)
Translating frames...
(1344, 256, 256, 3)
Translating frames...
(1373, 256, 256, 3)
Translating frames...
Tran

In [26]:
mean_of_all_trials = np.mean(trial_frames, axis=0)
print(np.shape(mean_of_all_trials))
mean_of_all_trials.tofile(hdd_directory+mouse+"_average.raw")

(160, 256, 256)


In [10]:
trial_frames.tofile(hdd_directory+mouse+"_all_trials.raw")

In [6]:
# Load the frames
def get_raw_frames(filename, channel):
    with open(filename, "rb") as file:
        raw_frames = np.fromfile(file, dtype=np.uint8)
        raw_frames = np.reshape(raw_frames, (int(raw_frames.shape[0]/(width*height*3)), height, width, 3))
        print(np.shape(raw_frames))
        
        
    return raw_frames[:, :, :, channel].astype(np.float32)


    
def get_dark_frames(green_frames):
    threshold = 5
    green_frames = np.reshape(green_frames, (green_frames.shape[0], green_frames.shape[1]*green_frames.shape[2]))
    temporal_means = np.mean(green_frames, axis=1)
    start_index = 0
    end_index = 0
    for mean, i in zip(temporal_means, range(0, temporal_means.shape[0]-1)):
        if mean < threshold and i < 1000:
            start_index= i
        elif mean < threshold and i >= 1000:
            end_index = i
            break

            
    return (start_index, end_index)

def cheby_filter(frames, low_limit, high_limit, frame_rate):
    nyq = frame_rate/2.0
    low_limit = low_limit/nyq
    high_limit = high_limit/nyq
    order = 4
    rp = 0.1
    Wn = [low_limit, high_limit]
    
    b, a = signal.cheby1(order, rp, Wn, 'bandpass', analog=False)
    print("Filtering...")

    #frames = frames.reshape(frames.shape[0], frames.shape[1]*frames.shape[2])
    #parallel_result = filter_pixel.map(frames[:])
    #parallel_result= np.asarray(parallel_result).reshape((frames.shape[0], height, width))
    frames = signal.filtfilt(b, a, frames, axis=0)
    #print("Shape!", parallel_result.shape)
    print("Done!")
    return frames

In [9]:
for file in data_files:
    frames = get_raw_frames(file, 1)

(1354, 256, 256, 3)
(1324, 256, 256, 3)
(1325, 256, 256, 3)


KeyboardInterrupt: 